<a href="https://flightspriceprediction-withstops.streamlit.app/">Flight_Price_prediction_with_stops_app </a>

In [1]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

from category_encoders import BinaryEncoder,HashingEncoder
from sklearn.preprocessing import OneHotEncoder

import sklearn
sklearn.set_config(transform_output="pandas")
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector,RFE

from sklearn.model_selection import cross_validate

from sklearn.model_selection import GridSearchCV
import joblib as jb

In [2]:
ML = pd.read_csv('cleaned_flights_with_stops_points.csv')

In [3]:
ML

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Price,day_of_journey,...,NDC,PAT,PNQ,RPR,STV,TRV,UDR,VGA,VNS,VTZ
0,Air India,2019-05-01,Kolkata,Banglore,05:50,13:15,7.42,2,7662,1,...,0,0,0,0,0,0,0,0,0,0
1,Jet Airways,2019-06-09,Delhi,Cochin,09:25,04:25,19.00,2,13882,9,...,0,0,0,0,0,0,0,0,0,0
2,IndiGo,2019-05-12,Kolkata,Banglore,18:05,23:30,5.42,1,6218,12,...,0,0,0,0,0,0,0,0,0,0
3,IndiGo,2019-03-01,Banglore,New Delhi,16:50,21:35,4.75,1,13302,1,...,0,0,0,0,0,0,0,0,0,0
4,SpiceJet,2019-06-24,Kolkata,Banglore,09:00,11:25,2.42,0,3873,24,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10315,Air Asia,2019-04-09,Kolkata,Banglore,19:55,22:25,2.50,0,4107,9,...,0,0,0,0,0,0,0,0,0,0
10316,Air India,2019-04-27,Kolkata,Banglore,20:45,23:20,2.58,0,4145,27,...,0,0,0,0,0,0,0,0,0,0
10317,Jet Airways,2019-04-27,Banglore,Delhi,08:20,11:20,3.00,0,7229,27,...,0,0,0,0,0,0,0,0,0,0
10318,Vistara,2019-03-01,Banglore,New Delhi,11:30,14:10,2.67,0,12648,1,...,0,0,0,0,0,0,0,0,0,0


# preparing to ML

###### drop useless columns in ML . 'stop_points','Total_Stops' will be dropped to avoid leakage and multi-collinearity

In [4]:
ML.drop(['Date_of_Journey','Dep_Time','Arrival_Time','stop_points','Total_Stops'],axis=1,inplace=True)

In [5]:
ML.duplicated().sum()

109

In [6]:
ML.drop_duplicates(inplace=True)

In [7]:
ML.reset_index(drop=True,inplace=True)

In [8]:
for col in ML.columns:
   
        print(f'{col} {ML[col].nunique()}')
    

Airline 12
Source 5
Destination 6
Duration 364
Price 1840
day_of_journey 10
month_of_journey 4
count_journey_days 3
dep_day_part 6
arrival_day_part 6
check-in baggage included 2
In-flight meal 2
Change airports 2
Business class 2
layover 4
 AMD  2
 ATQ  2
 BBI  2
 BDQ  2
 BHO  2
 BLR  2
 BOM  2
 CCU  2
 COK  2
 DED  2
 DEL  2
 GAU  2
 GOI  2
 GWL  2
 HBX  2
 HYD  2
 IDR  2
 IMF  2
 ISK  2
 IXA  2
 IXB  2
 IXC  2
 IXR  2
 IXU  2
 IXZ  2
 JAI  2
 JDH  2
 JLR  2
 KNU  2
 LKO  2
 MAA  2
 NAG  2
 NDC  2
 PAT  2
 PNQ  2
 RPR  2
 STV  2
 TRV  2
 UDR  2
 VGA  2
 VNS  2
 VTZ  2


In [9]:
for col in ML.columns:
    print(ML[col].value_counts(),'\n','_'*50)

Airline
Jet Airways                          3609
IndiGo                               1951
Air India                            1653
Multiple carriers                    1196
SpiceJet                              814
Vistara                               456
Air Asia                              315
GoAir                                 194
Multiple carriers Premium economy      13
Jet Airways Business                    6
Vistara Premium economy                 3
Trujet                                  1
Name: count, dtype: int64 
 __________________________________________________
Source
Delhi       4345
Kolkata     2814
Banglore    1995
Mumbai       689
Chennai      368
Name: count, dtype: int64 
 __________________________________________________
Destination
Cochin       4345
Banglore     2814
Delhi        1232
New Delhi     763
Hyderabad     689
Kolkata       368
Name: count, dtype: int64 
 __________________________________________________
Duration
2.83     517
1.50     380
2.75

In [10]:
ML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10211 entries, 0 to 10210
Data columns (total 57 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Airline                    10211 non-null  object 
 1   Source                     10211 non-null  object 
 2   Destination                10211 non-null  object 
 3   Duration                   10211 non-null  float64
 4   Price                      10211 non-null  int64  
 5   day_of_journey             10211 non-null  int64  
 6   month_of_journey           10211 non-null  int64  
 7   count_journey_days         10211 non-null  float64
 8   dep_day_part               10211 non-null  object 
 9   arrival_day_part           10211 non-null  object 
 10  check-in baggage included  10211 non-null  int64  
 11  In-flight meal             10211 non-null  int64  
 12  Change airports            10211 non-null  int64  
 13  Business class             10211 non-null  int

In [11]:
onh = ['Source','Destination' , 'dep_day_part' , 'arrival_day_part']

In [12]:
binary = ['Airline']

In [13]:
X = ML.drop('Price',axis=1)

In [14]:
X

,Airline,Source,Destination,Duration,day_of_journey,month_of_journey,count_journey_days,dep_day_part,arrival_day_part,check-in baggage included,...,NDC,PAT,PNQ,RPR,STV,TRV,UDR,VGA,VNS,VTZ
0,Air India,Kolkata,Banglore,7.42,1,5,0.0,early morning,afternoon,1,...,0,0,0,0,0,0,0,0,0,0
1,Jet Airways,Delhi,Cochin,19.00,9,6,1.0,morning,mid night,1,...,0,0,0,0,0,0,0,0,0,0
2,IndiGo,Kolkata,Banglore,5.42,12,5,0.0,evening,night,1,...,0,0,0,0,0,0,0,0,0,0
3,IndiGo,Banglore,New Delhi,4.75,1,3,0.0,afternoon,evening,1,...,0,0,0,0,0,0,0,0,0,0
4,SpiceJet,Kolkata,Banglore,2.42,24,6,0.0,morning,morning,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10206,Air Asia,Kolkata,Banglore,2.50,9,4,0.0,evening,evening,1,...,0,0,0,0,0,0,0,0,0,0
10207,Air India,Kolkata,Banglore,2.58,27,4,0.0,evening,night,1,...,0,0,0,0,0,0,0,0,0,0
10208,Jet Airways,Banglore,Delhi,3.00,27,4,0.0,morning,morning,1,...,0,0,0,0,0,0,0,0,0,0
10209,Vistara,Banglore,New Delhi,2.67,1,3,0.0,morning,afternoon,1,...,0,0,0,0,0,0,0,0,0,0


In [15]:
y= ML[['Price']]

In [16]:
y

,Price
0,7662
1,13882
2,6218
3,13302
4,3873
...,...
10206,4107
10207,4145
10208,7229
10209,12648


In [17]:
X.columns

Index(['Airline', 'Source', 'Destination', 'Duration', 'day_of_journey',
       'month_of_journey', 'count_journey_days', 'dep_day_part',
       'arrival_day_part', 'check-in baggage included', 'In-flight meal',
       'Change airports', 'Business class', 'layover', ' AMD ', ' ATQ ',
       ' BBI ', ' BDQ ', ' BHO ', ' BLR ', ' BOM ', ' CCU ', ' COK ', ' DED ',
       ' DEL ', ' GAU ', ' GOI ', ' GWL ', ' HBX ', ' HYD ', ' IDR ', ' IMF ',
       ' ISK ', ' IXA ', ' IXB ', ' IXC ', ' IXR ', ' IXU ', ' IXZ ', ' JAI ',
       ' JDH ', ' JLR ', ' KNU ', ' LKO ', ' MAA ', ' NAG ', ' NDC ', ' PAT ',
       ' PNQ ', ' RPR ', ' STV ', ' TRV ', ' UDR ', ' VGA ', ' VNS ', ' VTZ '],
      dtype='object')

In [18]:
X.describe()

,Duration,day_of_journey,month_of_journey,count_journey_days,check-in baggage included,In-flight meal,Change airports,Business class,layover,AMD,...,NDC,PAT,PNQ,RPR,STV,TRV,UDR,VGA,VNS,VTZ
count,10211.000000,10211.000000,10211.000000,10211.000000,10211.000000,10211.000000,10211.000000,10211.000000,10211.000000,10211.000000,...,10211.000000,10211.000000,10211.000000,10211.000000,10211.000000,10211.000000,10211.000000,10211.000000,10211.000000,10211.000000
mean,10.483595,13.443835,4.725198,0.394477,0.968857,0.817158,0.000686,0.000392,0.004113,0.022916,...,0.000098,0.000979,0.007345,0.001665,0.000196,0.002252,0.001959,0.002448,0.001077,0.000392
std,8.303614,8.485469,1.154598,0.495528,0.173712,0.386556,0.026175,0.019789,0.091685,0.149645,...,0.009896,0.031281,0.085392,0.040771,0.013995,0.047409,0.044216,0.049423,0.032806,0.019789
min,1.250000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.830000,6.000000,4.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,8.420000,12.000000,5.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,15.170000,21.000000,6.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,47.670000,27.000000,6.000000,2.000000,1.000000,1.000000,1.000000,1.000000,3.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [19]:
Airlines = ML['Airline'].unique()
Airlines

array(['Air India', 'Jet Airways', 'IndiGo', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [20]:
Sources = ML['Source'].unique()
Sources

array(['Kolkata', 'Delhi', 'Banglore', 'Chennai', 'Mumbai'], dtype=object)

In [21]:
ML[['Source','Destination']].value_counts()

Source    Destination
Delhi     Cochin         4345
Kolkata   Banglore       2814
Banglore  Delhi          1232
          New Delhi       763
Mumbai    Hyderabad       689
Chennai   Kolkata         368
Name: count, dtype: int64

In [22]:
ML[['Airline','Source','Destination']].value_counts()

Airline                            Source    Destination
Jet Airways                        Delhi     Cochin         1438
                                   Kolkata   Banglore       1250
Multiple carriers                  Delhi     Cochin         1196
IndiGo                             Delhi     Cochin          705
Air India                          Delhi     Cochin          703
                                   Kolkata   Banglore        501
IndiGo                             Kolkata   Banglore        405
Jet Airways                        Banglore  Delhi           370
IndiGo                             Banglore  Delhi           348
Jet Airways                        Banglore  New Delhi       337
SpiceJet                           Kolkata   Banglore        300
Jet Airways                        Mumbai    Hyderabad       214
IndiGo                             Mumbai    Hyderabad       195
Vistara                            Kolkata   Banglore        183
IndiGo                           

In [23]:
source_to_dest = list(ML[['Source','Destination']].value_counts().index)
source_to_dest

[('Delhi', 'Cochin'),
 ('Kolkata', 'Banglore'),
 ('Banglore', 'Delhi'),
 ('Banglore', 'New Delhi'),
 ('Mumbai', 'Hyderabad'),
 ('Chennai', 'Kolkata')]

In [24]:
Source_dest = dict()
for source in Sources:
    dest = []
    for s , d in source_to_dest:
        if source == s:
            dest.append(d)
    Source_dest[source] =dest

In [25]:
Source_dest

{'Kolkata': ['Banglore'],
 'Delhi': ['Cochin'],
 'Banglore': ['Delhi', 'New Delhi'],
 'Chennai': ['Kolkata'],
 'Mumbai': ['Hyderabad']}

In [26]:
Destinations = ML['Destination'].unique()
Destinations

array(['Banglore', 'Cochin', 'New Delhi', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

# Modeling

## Detecting which best model

In [27]:
scalers = [
    ('std scaler:', StandardScaler()), 
    ('MinMax scaler :', MinMaxScaler()), 
    ('Robust scaler: ', RobustScaler())
]

In [28]:
#models
models = [
    ('LinearR',LinearRegression()),      
    ('KNN', KNeighborsRegressor()),
    ('SVR', SVR()),

    ('DTR', DecisionTreeRegressor()),
    ('RFR', RandomForestRegressor()),
    ('ADA' ,AdaBoostRegressor()),
    ('GB',GradientBoostingRegressor()),
    
    ('XGB', XGBRegressor())]
    

In [29]:
# metrics and scores names
(sklearn.metrics.get_scorer_names())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',

In [30]:
result_models=[] # to append data of scores
for scaler_name, scaler in scalers:
    # print scaler name
    print(f'For {scaler_name} :')
    print('='*30)
    
    # Create the ColumnTransformer preprocessing
    preprocessor =ColumnTransformer(
        transformers=[
            ('binary_encoder', BinaryEncoder() ,binary),
            ('onh_encoder', OneHotEncoder(sparse_output=False,drop='first'), onh)
            ],
        remainder='passthrough'
         )
    
    for model_name, model in models:
        # print model name
        print(f"For {model_name}:")
        # Steps Creation
        stepss = list()
        
        stepss.append(('preprocessor', preprocessor))
        stepss.append((scaler_name, scaler))
        stepss.append((model_name, model))
        
        # Create the pipeline
        pipeline = Pipeline(steps=stepss)
        
        
        ## scoring
        # Perform cross-validation
        # Example evaluation metrics
        cv_results = cross_validate(pipeline, X, y, scoring={'R2':'r2'}
#                                                        'neg_mean_squared_error': 'neg_mean_squared_error',
#                                                        'neg_mean_absolute_error':'neg_mean_absolute_error',
#                                                        'neg_root_mean_squared_error':'neg_root_mean_squared_error',
#                                                                       }# interested scors and metrics
                                                             
                             , cv=5, return_train_score=True,n_jobs=-1)  # X and y are your input data and targets
        
        
        # Access the results
        test_r2 = round(cv_results['test_R2'].mean() , 5)
        train_r2 = round(cv_results['train_R2'].mean() , 5)
        print("test_R^2 :", test_r2)
        print("train_R^2:", train_r2)
        print('-'*50)

        result_models.append([test_r2,train_r2])
       # print('*' * 50)
    print('*' * 100)
    print('_' * 100)

For std scaler: :
For LinearR:
test_R^2 : -2.2145013176319132e+20
train_R^2: 0.67279
--------------------------------------------------
For KNN:
test_R^2 : 0.78751
train_R^2: 0.86268
--------------------------------------------------
For SVR:
test_R^2 : 0.01844
train_R^2: 0.01908
--------------------------------------------------
For DTR:
test_R^2 : 0.84448
train_R^2: 0.99497
--------------------------------------------------
For RFR:
test_R^2 : 0.89944
train_R^2: 0.98199
--------------------------------------------------
For ADA:
test_R^2 : 0.27385
train_R^2: 0.29198
--------------------------------------------------
For GB:
test_R^2 : 0.8228
train_R^2: 0.8415
--------------------------------------------------
For XGB:
test_R^2 : 0.90635
train_R^2: 0.95364
--------------------------------------------------
****************************************************************************************************
________________________________________________________________________________

In [31]:
result = pd.DataFrame(data=result_models,index=[
    'LinearR_std','KNN_std','SVR_std','DTR_std','RFR_std','ADA_std' ,'GB_std','XGB_std',
    'LinearR_minmax','KNN_minmax','SVR_minmax','DTR_minmax','RFR_minmax','ADA_minmax' ,'GB_minmax','XGB_minmax',
    'LinearR_robust','KNN_robust','SVR_robust','DTR_robust','RFR_robust','ADA_robust' ,'GB_robust','XGB_robust'],
                      columns=['test_R2' , 'train_R2'] )

In [32]:
result

,test_R2,train_R2
LinearR_std,-2.214501e+20,0.67279
KNN_std,7.875100e-01,0.86268
SVR_std,1.844000e-02,0.01908
DTR_std,8.444800e-01,0.99497
RFR_std,8.994400e-01,0.98199
ADA_std,2.738500e-01,0.29198
GB_std,8.228000e-01,0.84150
XGB_std,9.063500e-01,0.95364
LinearR_minmax,-7.896323e+19,0.66362
KNN_minmax,7.922000e-01,0.86546


In [33]:
result_models=[] # to append data of scores
for scaler_name, scaler in scalers:
    # print scaler name
    print(f'For {scaler_name} :')
    print('='*30)
    
    # Create the ColumnTransformer preprocessing
    preprocessor =ColumnTransformer(
        transformers=[
            ('binary_encoder', BinaryEncoder() ,binary),
            ('onh_encoder', OneHotEncoder(sparse_output=False,drop='first'), onh)
            ],
        remainder='passthrough'
         )
    
    for model_name, model in models:
        # print model name
        print(f"For {model_name}:")
        # Steps Creation
        stepss = list()
        
        stepss.append(('preprocessor', preprocessor))
        stepss.append((scaler_name, scaler))
        stepss.append((model_name, model))
        
        # Create the pipeline
        pipeline = Pipeline(steps=stepss)
        
        
        ## scoring
        # Perform cross-validation
        # Example evaluation metrics
        cv_results = cross_validate(pipeline, X, y, scoring={'R2':'r2',
                                                               'neg_mean_squared_error': 'neg_mean_squared_error',
                                                        'neg_mean_absolute_error':'neg_mean_absolute_error',
                                                        'neg_root_mean_squared_error':'neg_root_mean_squared_error'
                                                                       }# interested scors and metrics
                                                             
                                    , cv=5, return_train_score=True,n_jobs=-1)  # X and y are your input data and targets
        
        
        # Access the results
        test_r2 = cv_results['test_R2'].mean().round(5)
        train_r2 = cv_results['train_R2'].mean().round(5)
        print("test_R^2 :", test_r2)
        print("train_R^2:", train_r2)
        print('-'*50)
        test_neg_mean_squared_error = cv_results['test_neg_mean_squared_error'].mean().round(5)
        train_neg_mean_squared_error = cv_results['train_neg_mean_squared_error'].mean().round(5)
        print("test_neg_mean_squared_error :", test_neg_mean_squared_error)
        print("train_neg_mean_squared_error:", train_neg_mean_squared_error)
        print('-'*50)
        test_neg_mean_absolute_error = cv_results['test_neg_mean_absolute_error'].mean().round(5)
        train_neg_mean_absolute_error = cv_results['train_neg_mean_absolute_error'].mean().round(5)
        print("test_neg_mean_absolute_error :", test_neg_mean_absolute_error)
        print("train_neg_mean_absolute_error:", train_neg_mean_absolute_error)
        print('-'*50)
        test_neg_root_mean_squared_error = cv_results['test_neg_root_mean_squared_error'].mean().round(5)
        train_neg_root_mean_squared_error = cv_results['train_neg_root_mean_squared_error'].mean().round(5)
        print("test_neg_root_mean_squared_error :", test_neg_root_mean_squared_error)
        print("train_neg_root_mean_squared_error:", train_neg_root_mean_squared_error)
        print('-'*50)

        result_models.append([test_r2 , train_r2 , test_neg_mean_squared_error , train_neg_mean_squared_error ,
                              test_neg_mean_absolute_error , train_neg_mean_absolute_error ,
                              test_neg_root_mean_squared_error , train_neg_root_mean_squared_error])
        print('*' * 50)
    print('*' * 100)
    print('_' * 100)

For std scaler: :
For LinearR:
test_R^2 : -2.2145013176319132e+20
train_R^2: 0.67279
--------------------------------------------------
test_neg_mean_squared_error : -4.716470446207307e+27
train_neg_mean_squared_error: -6892425.2099
--------------------------------------------------
test_neg_mean_absolute_error : -1011415620180.7748
train_neg_mean_absolute_error: -1813.96036
--------------------------------------------------
test_neg_root_mean_squared_error : -45709209329094.19
train_neg_root_mean_squared_error: -2624.80555
--------------------------------------------------
**************************************************
For KNN:
test_R^2 : 0.78751
train_R^2: 0.86268
--------------------------------------------------
test_neg_mean_squared_error : -4474856.11713
train_neg_mean_squared_error: -2892898.09419
--------------------------------------------------
test_neg_mean_absolute_error : -1083.07815
train_neg_mean_absolute_error: -851.46694
--------------------------------------------

test_R^2 : 0.82289
train_R^2: 0.8415
--------------------------------------------------
test_neg_mean_squared_error : -3726674.17606
train_neg_mean_squared_error: -3338423.04871
--------------------------------------------------
test_neg_mean_absolute_error : -1295.20845
train_neg_mean_absolute_error: -1265.34606
--------------------------------------------------
test_neg_root_mean_squared_error : -1928.41868
train_neg_root_mean_squared_error: -1827.06352
--------------------------------------------------
**************************************************
For XGB:
test_R^2 : 0.90635
train_R^2: 0.95364
--------------------------------------------------
test_neg_mean_squared_error : -1972917.56156
train_neg_mean_squared_error: -976415.57408
--------------------------------------------------
test_neg_mean_absolute_error : -791.57057
train_neg_mean_absolute_error: -617.41083
--------------------------------------------------
test_neg_root_mean_squared_error : -1397.85455
train_neg_root_mea

In [34]:
scores = pd.DataFrame(data=result_models,index=[
    'LinearR_std','KNN_std','SVR_std','DTR_std','RFR_std','ADA_std' ,'GB_std','XGB_std',
    'LinearR_minmax','KNN_minmax','SVR_minmax','DTR_minmax','RFR_minmax','ADA_minmax' ,'GB_minmax','XGB_minmax',
    'LinearR_robust','KNN_robust','SVR_robust','DTR_robust','RFR_robust','ADA_robust' ,'GB_robust','XGB_robust'],
                      columns=['test_R2' , 'train_R2','test_neg_mean_squared_error' , 'train_neg_mean_squared_error' ,
                              'test_neg_mean_absolute_error' , 'train_neg_mean_absolute_error' ,
                              'test_neg_root_mean_squared_error' , 'train_neg_root_mean_squared_error'] )

In [35]:
scores

,test_R2,train_R2,test_neg_mean_squared_error,train_neg_mean_squared_error,test_neg_mean_absolute_error,train_neg_mean_absolute_error,test_neg_root_mean_squared_error,train_neg_root_mean_squared_error
LinearR_std,-2.214501e+20,0.67279,-4.716470e+27,-6.892425e+06,-1.011416e+12,-1813.96036,-4.570921e+13,-2624.80555
KNN_std,7.875100e-01,0.86268,-4.474856e+06,-2.892898e+06,-1.083078e+03,-851.46694,-2.113628e+03,-1700.65816
SVR_std,1.844000e-02,0.01908,-2.067533e+07,-2.066528e+07,-3.487502e+03,-3485.80366,-4.545643e+03,-4545.84357
DTR_std,8.387500e-01,0.99497,-3.391597e+06,-1.058085e+05,-7.390000e+02,-52.16883,-1.831815e+03,-325.02814
RFR_std,8.981400e-01,0.98197,-2.145873e+06,-3.796931e+05,-6.718407e+02,-270.27002,-1.456206e+03,-615.95823
ADA_std,1.736900e-01,0.19881,-1.737129e+07,-1.687326e+07,-3.391713e+03,-3365.36688,-4.118275e+03,-4055.70223
GB_std,8.213300e-01,0.84150,-3.760431e+06,-3.338423e+06,-1.295349e+03,-1265.34606,-1.935975e+03,-1827.06352
XGB_std,9.063500e-01,0.95364,-1.972918e+06,-9.764156e+05,-7.915706e+02,-617.41083,-1.397855e+03,-987.62135
LinearR_minmax,-7.896323e+19,0.66362,-1.661969e+27,-7.084374e+06,-4.616663e+11,-1842.05718,-2.030134e+13,-2660.51067
KNN_minmax,7.922000e-01,0.86546,-4.387302e+06,-2.834624e+06,-1.063061e+03,-830.88413,-2.089977e+03,-1682.95056


# Best models with feature selection

In [36]:
models = [('RFR', RandomForestRegressor()),
            ('XGB', XGBRegressor())]
    

In [37]:
feature_selector = RFE(XGBRegressor(),n_features_to_select=50) 

In [38]:
result_models=[] # to append data of scores

# Create the ColumnTransformer preprocessing
preprocessor =ColumnTransformer(
    transformers=[
        ('binary_encoder', BinaryEncoder() ,binary),
        ('onh_encoder', OneHotEncoder(sparse_output=False,drop='first'), onh)
        ],
    remainder='passthrough'
     )

for model_name, model in models:
    # print model name
    print(f"For {model_name}:")
    # Steps Creation
    stepss = list()

    stepss.append(('preprocessor', preprocessor))
    stepss.append(('scaler', RobustScaler()))
    stepss.append(('feature_selector', feature_selector))
    stepss.append((model_name, model))

    # Create the pipeline
    pipeline = Pipeline(steps=stepss)


    ## scoring
    # Perform cross-validation
    # Example evaluation metrics
    cv_results = cross_validate(pipeline, X, y, scoring={'R2':'r2'}
#                                                        'neg_mean_squared_error': 'neg_mean_squared_error',
#                                                        'neg_mean_absolute_error':'neg_mean_absolute_error',
#                                                        'neg_root_mean_squared_error':'neg_root_mean_squared_error',
#                                                                       }# interested scors and metrics

                         , cv=5, return_train_score=True,n_jobs=-1)  # X and y are your input data and targets


    # Access the results
    test_r2 = round(cv_results['test_R2'].mean() , 5)
    train_r2 = round(cv_results['train_R2'].mean() , 5)
    print("test_R^2 :", test_r2)
    print("train_R^2:", train_r2)
    print('-'*50)

    result_models.append([test_r2,train_r2])
   # print('*' * 50)

For RFR:
test_R^2 : 0.89629
train_R^2: 0.98155
--------------------------------------------------
For XGB:
test_R^2 : 0.9053
train_R^2: 0.95911
--------------------------------------------------


- XGBoost r2 score decreased with feature selection 

HyperParameter Tuning 
--------------------------------------------------

In [62]:
# initialize RFE for feature selection
feature_selector = RFE(XGBRegressor(),n_features_to_select=50) 

In [63]:
# Create the ColumnTransformer preprocessing
encoding = ColumnTransformer(
        transformers=[
            ('binary_encoder', BinaryEncoder() , binary),
            ('onh_encoder', OneHotEncoder(sparse_output=False,drop='first'), onh)],
        remainder='passthrough'
         )

In [64]:
# steps and pipeline for HyperParameter Tuning
steps=[]
steps.append(('encoding', encoding))
  
steps.append(('scaler', StandardScaler()))
steps.append(('feature selection', feature_selector))
steps.append(('RFR', RandomForestRegressor()))
pipee=Pipeline(steps=steps)

In [65]:
param_grid = {
'RFR__max_depth': [None],
'RFR__min_samples_leaf': [1],
'RFR__min_samples_split': [6],

'RFR__n_estimators': [500,700,1000] #500,700,800]

}

In [66]:
# Create GridSearchCV instance
grid_search = GridSearchCV(estimator=pipee, param_grid=param_grid, cv=5, scoring='r2'
                                                             , n_jobs=-1, return_train_score=True)

# Fit the pipeline with GridSearch to the data
grid_search.fit(X, y)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'RFR__max_depth': None, 'RFR__min_samples_leaf': 1, 'RFR__min_samples_split': 6, 'RFR__n_estimators': 1000}
Best Score: 0.9015438155321405


In [67]:
# Get the mean test score and mean train score for the best estimator
mean_test_score = grid_search.cv_results_['mean_test_score'][grid_search.best_index_]
mean_train_score = grid_search.cv_results_['mean_train_score'][grid_search.best_index_]

print("Mean Test Score:", mean_test_score)
print("Mean Train Score:", mean_train_score)

Mean Test Score: 0.9015438155321405
Mean Train Score: 0.9643147066283049


* **XGBoost**

In [68]:
# Create the ColumnTransformer preprocessing
encoding = ColumnTransformer(
        transformers=[
            ('binary_encoder', BinaryEncoder() , binary),
            ('onh_encoder', OneHotEncoder(sparse_output=False,drop='first'), onh)],
        remainder='passthrough'
         )

In [69]:
# steps and pipeline for HyperParameter Tuning
steps=[]
steps.append(('encoding', encoding))
  
steps.append(('scaler', RobustScaler()))
steps.append(('XGB', XGBRegressor()))
pipe=Pipeline(steps=steps)

In [84]:
param_grid = {
 'XGB__learning_rate':[0.1],
 'XGB__n_estimators':[1200,1300,400],
 'XGB__max_depth':[6],
 'XGB__min_child_weight':[3,4],
 'XGB__gamma':[2],#}#,4]#,5,6]}
  'XGB__subsample':[2,1],
  'XGB__colsample_bytree':[0.8]}#,
#  'XGB__objective' :['binary:logistic'],
#  'XGB__nthread':[4],
#  'XGB__scale_pos_weight':[1],
#  'XGB__seed':[27]}'''

In [85]:
# Create GridSearchCV instance
grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=5, scoring='r2'
                                                             , n_jobs=-1, return_train_score=True)

# Fit the pipeline with GridSearch to the data
grid_search.fit(X, y)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'XGB__colsample_bytree': 0.8, 'XGB__gamma': 2, 'XGB__learning_rate': 0.1, 'XGB__max_depth': 6, 'XGB__min_child_weight': 3, 'XGB__n_estimators': 1200, 'XGB__subsample': 1}
Best Score: 0.916581315284206


In [86]:
# Get the mean test score and mean train score for the best estimator
mean_test_score = grid_search.cv_results_['mean_test_score'][grid_search.best_index_]
mean_train_score = grid_search.cv_results_['mean_train_score'][grid_search.best_index_]

print("Mean Test Score:", mean_test_score)
print("Mean Train Score:", mean_train_score)

Mean Test Score: 0.916581315284206
Mean Train Score: 0.9761638021036688


# Saving model with its best parameters

In [87]:
Model = grid_search.best_estimator_

In [88]:
Model.fit(X,y)

Pipeline(steps=[('encoding',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('binary_encoder',
                                                  BinaryEncoder(),
                                                  ['Airline']),
                                                 ('onh_encoder',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['Source', 'Destination',
                                                   'dep_day_part',
                                                   'arrival_day_part'])])),
                ('scaler', RobustScaler()),
                ('XGB',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              col...
                              feature_types=None, gamma=2, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=6, max_leaves=None, min_child_weight=3,
                              missing=nan, monotone_constraints=None,
                              multi_strategy=None, n_estimators=1200,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=None, ...))])

# scoring best params

In [89]:
# Create the ColumnTransformer preprocessing
encoding = ColumnTransformer(
        transformers=[
            ('binary_encoder', BinaryEncoder().set_output(transform="pandas") , binary),
            ('onh_encoder', OneHotEncoder(sparse_output=False,drop='first'), onh)],
        remainder='passthrough'
         )

In [90]:
# steps and pipeline for HyperParameter Tuning
steps=[]
steps.append(('encoding', encoding))
  
steps.append(('scaler', RobustScaler()))
steps.append(('XGB', XGBRegressor(max_depth=6 , min_child_weight=3 , gamma=2 , n_estimators=1200 , subsample= 1,
                                  colsample_bytree= 0.8,learning_rate=.1)))
pipe=Pipeline(steps=steps)

In [91]:
'''# steps and pipeline for HyperParameter Tuning
steps=[]
steps.append(('encoding', encoding))
  
steps.append(('scaler', RobustScaler()))
steps.append(('RFR', RandomForestRegressor(max_depth=None, min_samples_leaf=1, min_samples_split=6, n_estimators=700)))
pipe=Pipeline(steps=steps)'''

"# steps and pipeline for HyperParameter Tuning\nsteps=[]\nsteps.append(('encoding', encoding))\n  \nsteps.append(('scaler', RobustScaler()))\nsteps.append(('RFR', RandomForestRegressor(max_depth=None, min_samples_leaf=1, min_samples_split=6, n_estimators=700)))\npipe=Pipeline(steps=steps)"

In [92]:
cv_results = cross_validate(pipe, X, y, scoring={'R2':'r2',
                                                'neg_mean_squared_error': 'neg_mean_squared_error',
                                                'neg_mean_absolute_error':'neg_mean_absolute_error',
                                                'neg_root_mean_squared_error':'neg_root_mean_squared_error'}# interested scors and metrics

                                , cv=5, return_train_score=True,n_jobs=-1)

In [93]:
test_r2 = cv_results['test_R2'].mean().round(5)
train_r2 = cv_results['train_R2'].mean().round(5)
print("test_R^2 :", test_r2)
print("train_R^2:", train_r2)
print('-'*50)
test_neg_mean_squared_error = cv_results['test_neg_mean_squared_error'].mean().round(5)
train_neg_mean_squared_error = cv_results['train_neg_mean_squared_error'].mean().round(5)
print("test_neg_mean_squared_error :", test_neg_mean_squared_error)
print("train_neg_mean_squared_error:", train_neg_mean_squared_error)
print('-'*50)
test_neg_mean_absolute_error = cv_results['test_neg_mean_absolute_error'].mean().round(5)
train_neg_mean_absolute_error = cv_results['train_neg_mean_absolute_error'].mean().round(5)
print("test_neg_mean_absolute_error :", test_neg_mean_absolute_error)
print("train_neg_mean_absolute_error:", train_neg_mean_absolute_error)
print('-'*50)
test_neg_root_mean_squared_error = cv_results['test_neg_root_mean_squared_error'].mean().round(5)
train_neg_root_mean_squared_error = cv_results['train_neg_root_mean_squared_error'].mean().round(5)
print("test_neg_root_mean_squared_error :", test_neg_root_mean_squared_error)
print("train_neg_root_mean_squared_error:", train_neg_root_mean_squared_error)
print('-'*50)


test_R^2 : 0.91658
train_R^2: 0.97616
--------------------------------------------------
test_neg_mean_squared_error : -1758564.95422
train_neg_mean_squared_error: -502192.43972
--------------------------------------------------
test_neg_mean_absolute_error : -690.4879
train_neg_mean_absolute_error: -410.91729
--------------------------------------------------
test_neg_root_mean_squared_error : -1319.63697
train_neg_root_mean_squared_error: -708.26293
--------------------------------------------------


* **These are best scores for the Model**

## Saving important objects For Deploying

In [94]:
jb.dump(Model,"Model_with_stops.pkl")

['Model_with_stops.pkl']

In [60]:
jb.dump(X.columns,"Columns.pkl")

['Columns.pkl']

In [61]:
jb.dump(Airlines ,"Airlines.pkl")
jb.dump(Sources ,"Sources.pkl")
#jb.dump(Destinations ,"Destinations.pkl")
#jb.dump(Source_to_dest ,"Source_to_dest.pkl")
jb.dump(Source_dest ,"Source_dest.pkl")


['Source_dest.pkl']

<a href="https://flightspriceprediction-withstops.streamlit.app/">Flight_Price_prediction_with_stops_app </a>